Basic code for antibody-based simulations
===========

## basic imports ##

In [2]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from mdtraj.reporters import *
from functional import seq

from pathlib import Path
import sys
from sys import stdout
import inspect
from pycomfort.files import *

In [3]:
debug_local = True#False
local = Path("..").resolve()
code = local / "mm"
data  = local / "data"
input = data / "input"
output = Path("/media/antonkulaga/Elements/molecular_dynamics")

if debug_local and code.exists():
    sys.path.insert(0, code.as_posix())
    print("extending pathes with local mm")
    print(sys.path)
    %load_ext autoreload
    %autoreload 2

extending pathes with local mm
['/data/sources/antibody-mm/mm', '/data/sources/antibody-mm/notebooks', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python39.zip', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/lib-dynload', '', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [4]:
children(input).for_each(print)


/data/sources/antibody-mm/data/input/RA
/data/sources/antibody-mm/data/input/1ADQ


In [5]:
tprint(output)

molecular_dynamics
	.~lock.FIXED_1_35744_H_112_energy_-265.2682_log.tsv#
	FIXED_1_35744_H_112_energy_-265.2682.dcd
	FIXED_1_35744_H_112_energy_-265.2682_log.h5
	FIXED_1_35744_H_112_energy_-265.2682_log.tsv
	FIXED_1_35744_H_112_energy_-265.2682_results.chk
	FIXED_1_35744_H_1_energy_-254.21349.dcd
	FIXED_1_35744_H_1_energy_-254.21349_log.h5
	FIXED_1_35744_H_1_energy_-254.21349_log.tsv
	FIXED_1_35744_H_1_energy_-254.21349_results.chk
	FIXED_1_35744_H_64_energy_-263.5477.dcd
	FIXED_1_35744_H_64_energy_-263.5477_log.h5
	FIXED_1_35744_H_64_energy_-263.5477_log.tsv
	FIXED_1_35744_H_64_energy_-263.5477_results.chk
	S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains
		FIXED_1_35744_H_112_energy_-265.2682.dcd
		FIXED_1_35744_H_112_energy_-265.2682_log.h5
		FIXED_1_35744_H_112_energy_-265.2682_log.tsv
		FIXED_1_35744_H_112_energy_-265.2682_results.chk
		FIXED_1_35744_H_1_energy_-254.21349.dcd
		FIXED_1_35744_H_1_energy_-254.21349_log.h5
		FIXED_1_35744_H_1_energy_-254.21349_log.tsv
		FIXED_1_35744_H_1_ener

## Setting simulation parameters ##

In [5]:
def make_simulation(path: Path, delete_water: bool = True, add_hydrogens: bool = True):
    pdb = PDBFile(path.as_posix())
    modeller = Modeller(pdb.topology, pdb.positions)
    forcefield = ForceField('amber14/protein.ff14SB.xml',  'implicit/gbn2.xml')
    temperature = 310 * kelvin
    integrator = LangevinMiddleIntegrator(temperature, 1 / picosecond, 0.002*picoseconds)
    if delete_water:
        modeller.deleteWater()
    if add_hydrogens:
        modeller.addHydrogens(forcefield)
        #forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
    system = forcefield.createSystem(modeller.topology, 
        nonbondedMethod=CutoffNonPeriodic,
        nonbondedCutoff=1.2*nanometer,
        switchDistance=1.0*nanometer, 
        constraints=HBonds
        #implicitSolvent=app.GBn2, implicitSolventSaltConc=0.15*moles/liter
        )
    simulation = Simulation(pdb.topology, system, integrator)
    simulation.context.setPositions(modeller.positions)
    simulation.minimizeEnergy()
    return simulation

In [6]:
def with_reporters(simulation: Simulation, pdb: Path, where: Path = output,
                 report_interval: int = 1000, checkpoint_interval: int = 10000, 
                 #report_pdb: bool = True #pdb reporter can be heavy
               ): 
    #simulation.reporters.append(PDBReporter((where / f'{pdb.stem}.dcd').as_posix(), report_interval))
    if not where.exists():
        where.mkdir()    
    simulation.reporters.append(DCDReporter((where / f'{pdb.stem}.dcd').as_posix(), report_interval))
    simulation.reporters.append(StateDataReporter((where / f'{pdb.stem}_log.tsv').as_posix(), report_interval, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, step=True,temperature=True, separator='\t'))
    simulation.reporters.append(HDF5Reporter((where / f'{pdb.stem}_log.h5').as_posix(), report_interval, coordinates=True, time=True, cell=True, potentialEnergy=True, kineticEnergy=True, temperature=True))
    simulation.reporters.append(CheckpointReporter((where / f'{pdb.stem}_results.chk').as_posix(), checkpoint_interval))
    return simulation

In [7]:
def save_results(sim: Simulation, pdb: Path, where: Path = output) -> Simulation:
    if not where.exists():
        where.mkdir()
    finalpositions = sim.context.getState(getPositions=True).getPositions()
    PDBFile.writeFile(sim.topology, finalpositions, open((where / f'{pdb.stem}.pdb').as_posix(), 'w'))
    sim.saveState((where/ f'{pdb.stem}_results.xml').as_posix())
    sim.saveCheckpoint((where / f'{pdb.stem}_results.chk').as_posix())
    return sim

### Loading folders and files ###

In [8]:
docking_results = Path("/data/samples/docking/RA/results")
best_docking_results = docking_results / "best"
tprint(best_docking_results)


best
	S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains
		3_17806_H_5_energy_-238.0523.pdb
		3_17806_H_35_energy_-224.0373.pdb
		FIXED_1_35744_H_64_energy_-263.5477.pdb
		FIXED_1_35744_H_1_energy_-254.21349.pdb
		1_35744_H_64_energy_-263.5477.pdb
		1_35744_H_1_energy_-254.21349.pdb
		2_27457_H_257_energy_-248.7721.pdb
		FIXED_1_35744_H_112_energy_-265.2682.pdb
		1_35744_H_112_energy_-265.2682.pdb
		2_27457_H_253_energy_-246.08788.pdb
		2_27457_H_244_energy_-284.44533.pdb
		3_17806_H_24_energy_-221.3707.pdb
	S5205Nr1-P2_PBMC_L_top_30_heavy_chains
		1_27244_H_30_energy_-263.64684.pdb
		1_27244_H_23_energy_-251.4212.pdb
		1_27244_H_297_energy_-260.30115.pdb


In [9]:
p2_auto = best_docking_results / "S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains"
p2_pbmc = best_docking_results / "S5205Nr1-P2_PBMC_L_top_30_heavy_chains"
tprint(p2_auto)

S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains
	3_17806_H_5_energy_-238.0523.pdb
	3_17806_H_35_energy_-224.0373.pdb
	FIXED_1_35744_H_64_energy_-263.5477.pdb
	FIXED_1_35744_H_1_energy_-254.21349.pdb
	1_35744_H_64_energy_-263.5477.pdb
	1_35744_H_1_energy_-254.21349.pdb
	2_27457_H_257_energy_-248.7721.pdb
	FIXED_1_35744_H_112_energy_-265.2682.pdb
	1_35744_H_112_energy_-265.2682.pdb
	2_27457_H_253_energy_-246.08788.pdb
	2_27457_H_244_energy_-284.44533.pdb
	3_17806_H_24_energy_-221.3707.pdb


Selecting the file
===================

In [10]:
selected_group = p2_auto
selected_pdb = "FIXED_1_35744_H_64_energy_-263.5477.pdb"


In [11]:
pdb = p2_auto / selected_pdb
where = output / selected_group.name
where.mkdir(parents=True, exist_ok=True)

### Making simulation

In [12]:
sim = with_reporters(make_simulation(pdb), pdb, where )
sim

## Running the simulation ##

In [13]:
#steps =  50000
ten_nano = 50000000 # to get 10 nanoseconds 
steps =  ten_nano * 5

In [ ]:
sim.step(steps)

/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting xyz dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


In [14]:
save_results(sim, where)

## Analyzing results

In [16]:
print(where.as_posix())
tprint(where)

/data/sources/antibody-mm/data/output/S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains
S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains
	FIXED_1_35744_H_112_energy_-265.2682_log.tsv
	FIXED_1_35744_H_112_energy_-265.2682_log.h5
	FIXED_1_35744_H_112_energy_-265.2682.dcd
	FIXED_1_35744_H_112_energy_-265.2682_results.chk
